In [1]:
from multiprocessing.sharedctypes import Value
import radix as r
import pandas as pd
import numpy as np
import ipaddress as ip
import time

In [3]:
data = pd.read_csv("../Data/ip_data.csv", na_filter=False)
print(data)

                INFOBLOX_TABLE  ROUTING_TABLE
0                   3.0.0.3/32  10.39.42.0/23
1      3.0.0.0/255.255.255.255  10.39.42.0/23
2                  3.0.0.64/26  10.39.44.0/22
3                   3.0.1.0/24  10.39.48.0/22
4                4.79.90.12/30  10.39.52.0/22
...                        ...            ...
94406         239.195.192.0/19               
94407         239.195.224.0/19               
94408           239.196.0.0/18               
94409          239.196.64.0/18               
94410         239.196.128.0/17               

[94411 rows x 2 columns]


In [9]:
class IPAM:
    def __init__(self) -> None:
        self.data = pd.read_csv("../Data/ip_data.csv", na_filter=False)

    def main(self) -> None:
        left_Tree = r.Radix()
        right_Tree = r.Radix()

        # Self Overlap
        for index, row in self.data.iterrows():
            if self.data["INFOBLOX_TABLE"][index]:
                try:
                    curr_network = str(ip.ip_network(
                        self.data["INFOBLOX_TABLE"][index]))
                except ValueError:
                    print(
                        f"The row : { self.data['INFOBLOX_TABLE'][index] } is not in the correct format")
                else:
                    if curr_network:
                        new_node = left_Tree.search_best(curr_network)
                        if not new_node:
                            curr_node = left_Tree.add(curr_network)
                            self.data.at[index,
                                         "SELF_OVERLAP_A"] = "NO_OVERLAP"
                            curr_node.data["INDEX"] = index
                        else:
                            self.data.at[index,
                                         "SELF_OVERLAP_A"] = new_node.data["INDEX"]

        for index, row in self.data.iterrows():
            if self.data["ROUTING_TABLE"][index]:
                try:
                    curr_network = str(ip.ip_network(
                        self.data["ROUTING_TABLE"][index]))
                except ValueError:
                    print(
                        f"The row : { self.data['ROUTING_TABLE'][index] } is not in the correct format")
                else:
                    if curr_network:
                        new_node = right_Tree.search_best(curr_network)
                        if not new_node:
                            curr_node = right_Tree.add(curr_network)
                            self.data.at[index,
                                         "SELF_OVERLAP_B"] = "NO_OVERLAP"
                            curr_node.data["INDEX"] = index
                        else:
                            self.data.at[index,
                                         "SELF_OVERLAP_B"] = new_node.data["INDEX"]

        # CROSS OVERLAP
        for index, row in self.data.iterrows():
            if self.data["ROUTING_TABLE"][index]:
                try:
                    curr_network = str(ip.ip_network(
                        self.data["ROUTING_TABLE"][index]))
                except ValueError:
                    print(
                        f"The row : { self.data['ROUTING_TABLE'][index] } is not in the correct format")
                else:
                    if curr_network:
                        new_node = left_Tree.search_best(curr_network)
                        if not new_node:
                            curr_node = left_Tree.add(curr_network)
                            self.data.at[index,
                                         "CROSS_OVERLAP_B_2_A"] = "NO_OVERLAP"
                            curr_node.data["INDEX"] = index
                        else:
                            self.data.at[index,
                                         "CROSS_OVERLAP_B_2_A"] = new_node.data["INDEX"]

        for index, row in self.data.iterrows():
            if self.data["INFOBLOX_TABLE"][index]:
                try:
                    curr_network = str(ip.ip_network(
                        self.data["INFOBLOX_TABLE"][index]))
                except ValueError:
                    print(
                        f"The row : { self.data['INFOBLOX_TABLE'][index] } is not in the correct format")
                else:
                    if curr_network:
                        new_node = right_Tree.search_best(curr_network)
                        if not new_node:
                            curr_node = right_Tree.add(curr_network)
                            self.data.at[index,
                                         "CROSS_OVERLAP_A_2_B"] = "NO_OVERLAP"
                            curr_node.data["INDEX"] = index
                        else:
                            self.data.at[index,
                                         "CROSS_OVERLAP_A_2_B"] = new_node.data["INDEX"]

        self.data.to_csv("../Data/out/Output.csv")


In [10]:
start = time.perf_counter()
_ipam = IPAM()
_ipam.main()
end = time.perf_counter()
print(f"The Whole script completed in {end-start:0.4f} seconds")


PermissionError: [Errno 13] Permission denied: '../Data/out/Output.csv'